In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_csv('train.csv',nrows=300000)
print(data.shape)
data.head()

(300000, 304)


,row_id,time_id,investment_id,target,f_0,f_1,f_2,f_3,f_4,f_5,...,f_290,f_291,f_292,f_293,f_294,f_295,f_296,f_297,f_298,f_299
0,0_1,0,1,-0.300875,0.932573,0.113691,-0.402206,0.378386,-0.203938,-0.413469,...,0.366028,-1.095620,0.200075,0.819155,0.941183,-0.086764,-1.087009,-1.044826,-0.287605,0.321566
1,0_2,0,2,-0.231040,0.810802,-0.514115,0.742368,-0.616673,-0.194255,1.771210,...,-0.154193,0.912726,-0.734579,0.819155,0.941183,-0.387617,-1.087009,-0.929529,-0.974060,-0.343624
2,0_6,0,6,0.568807,0.393974,0.615937,0.567806,-0.607963,0.068883,-1.083155,...,-0.138020,0.912726,-0.551904,-1.220772,-1.060166,-0.219097,-1.087009,-0.612428,-0.113944,0.243608
3,0_7,0,7,-1.064780,-2.343535,-0.011870,1.874606,-0.606346,-0.586827,-0.815737,...,0.382201,0.912726,-0.266359,-1.220772,0.941183,-0.609113,0.104928,-0.783423,1.151730,-0.773309
4,0_8,0,8,-0.531940,0.842057,-0.262993,2.330030,-0.583422,-0.618392,-0.742814,...,-0.170365,0.912726,-0.741355,-1.220772,0.941183,-0.588445,0.104928,0.753279,1.345611,-0.737624


In [4]:
# use validation split approach for training and testing data
target = data.pop('target')
x_train, x_test, y_train, y_test = train_test_split(data,target)

In [5]:
# remove id columns from training data
x_train_row_id = x_train.pop('row_id')
x_train_time_id = x_train.pop('time_id')
x_train_investment_id = x_train.pop('investment_id')

In [6]:
# remove id columns from training data
x_test_row_id = x_test.pop('row_id')
x_test_time_id = x_test.pop('time_id')
x_test_investment_id = x_test.pop('investment_id')

# Deep Neural Network

In [8]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import BatchNormalization, Dense, Dropout
from tensorflow.keras import Sequential
from sklearn.metrics import mean_squared_error

In [9]:
def create_dnn(nodes,layers,dropout,activation):
    model = Sequential()
    
    for i in list(range(1,layers+1)):
        if i == 1:
            model.add(Dense(nodes, input_dim = (x_train.shape[1]), activation = 'swish'))
            model.add(BatchNormalization())
            model.add(Dropout(0.3))
        else:
            model.add(Dense(i/2, activation='swish'))
            model.add(BatchNormalization())
            model.add(Dropout(0.3))

    model.add(BatchNormalization())
    model.add(Dense(1))
    model.compile(loss = 'mse', optimizer = 'adam', metrics = ['mae'])
    return model

In [10]:
results = {}
models = {}
for i in [512,256,128]:
    for j in [5,4,3]:
                model = create_dnn(i,j,0.3,'swish')
                model.fit(x_train, y_train, epochs = 40, batch_size = 4096, verbose = 1)
                
                y_pred = model.predict(x_test)
                error = mean_squared_error(y_test,y_pred)
                
                comb = str(i)+'-'+str(j)
                results[comb]= error
                models[comb] = model

Epoch 1/40
55/55 [==============================] - 3s 57ms/step - loss: 1.6009 - mae: 0.8362
Epoch 2/40
55/55 [==============================] - 3s 58ms/step - loss: 1.3267 - mae: 0.8271
Epoch 3/40
55/55 [==============================] - 3s 60ms/step - loss: 1.1789 - mae: 0.7852
Epoch 4/40
55/55 [==============================] - 3s 58ms/step - loss: 1.1226 - mae: 0.7587
Epoch 5/40
55/55 [==============================] - 3s 59ms/step - loss: 1.0814 - mae: 0.7399
Epoch 6/40
55/55 [==============================] - 3s 58ms/step - loss: 1.0545 - mae: 0.7285
Epoch 7/40
55/55 [==============================] - 3s 58ms/step - loss: 1.0274 - mae: 0.7160
Epoch 8/40
55/55 [==============================] - 3s 60ms/step - loss: 1.0058 - mae: 0.7066
Epoch 9/40
55/55 [==============================] - 3s 59ms/step - loss: 0.9842 - mae: 0.6964
Epoch 10/40
55/55 [==============================] - 3s 59ms/step - loss: 0.9723 - mae: 0.6926
Epoch 11/40
55/55 [==============================] - 3s 60m

In [28]:
for i in results.items():
    print(i)

('512-5', 0.8742649160363306)
('512-4', 7.032016064726037)
('512-3', 0.8670515338352196)
('256-5', 0.8775636687789832)
('256-4', 0.9220974353537369)
('256-3', 0.8593925899867325)
('128-5', 0.8682550885602324)
('128-4', 0.8703019781825282)
('128-3', 0.8621566736238333)


# Random Forest

In [30]:
from sklearn.ensemble import RandomForestRegressor

results_RF = {}
models_RF = {}

for i in [2,5,10]:
    model= RandomForestRegressor(max_depth=i,n_estimators=17)
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    error = mean_squared_error(y_test,y_pred)
    comb = str(i)
    results_RF[comb]= error
    models[comb] = model

In [31]:
results_RF

{'2': 0.8774245872809612, '5': 0.8741895983577129, '10': 0.8708021730451622}

# Ensemble

In [36]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

mod1 = GradientBoostingRegressor(n_estimators=10)
mod2 = RandomForestRegressor(n_estimators=10)
mod3 = LinearRegression()
ensemble = VotingRegressor(estimators=[('gb', mod1), ('rf', mod2), ('lr', mod3)])
ensemble_model = ensemble.fit(x_train, y_train)

In [38]:
y_pred = ensemble_model.predict(x_test)
error = mean_squared_error(y_test,y_pred)
error

0.8632789096469304

# LSTM

In [39]:
data_full = pd.read_csv('train.csv')
print(data.shape)
data.head()

(300000, 303)


,row_id,time_id,investment_id,f_0,f_1,f_2,f_3,f_4,f_5,f_6,...,f_290,f_291,f_292,f_293,f_294,f_295,f_296,f_297,f_298,f_299
0,0_1,0,1,0.932573,0.113691,-0.402206,0.378386,-0.203938,-0.413469,0.965623,...,0.366028,-1.095620,0.200075,0.819155,0.941183,-0.086764,-1.087009,-1.044826,-0.287605,0.321566
1,0_2,0,2,0.810802,-0.514115,0.742368,-0.616673,-0.194255,1.771210,1.428127,...,-0.154193,0.912726,-0.734579,0.819155,0.941183,-0.387617,-1.087009,-0.929529,-0.974060,-0.343624
2,0_6,0,6,0.393974,0.615937,0.567806,-0.607963,0.068883,-1.083155,0.979656,...,-0.138020,0.912726,-0.551904,-1.220772,-1.060166,-0.219097,-1.087009,-0.612428,-0.113944,0.243608
3,0_7,0,7,-2.343535,-0.011870,1.874606,-0.606346,-0.586827,-0.815737,0.778096,...,0.382201,0.912726,-0.266359,-1.220772,0.941183,-0.609113,0.104928,-0.783423,1.151730,-0.773309
4,0_8,0,8,0.842057,-0.262993,2.330030,-0.583422,-0.618392,-0.742814,-0.946789,...,-0.170365,0.912726,-0.741355,-1.220772,0.941183,-0.588445,0.104928,0.753279,1.345611,-0.737624
